### Projeto Consulta Juridica

Criando o Navegador

In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
import time

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

Abrindo a página

In [104]:
import os

caminho = os.getcwd()
arquivo = caminho + r'\index.html'
navegador.get(arquivo)

Lendo o arquivo com os processos a serem consultados

In [105]:
import pandas as pd

processos_df = pd.read_excel(r'Processos.xlsx')
display(processos_df)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


Preenchendo formulario e consultando os processos

In [ ]:
from selenium.webdriver import ActionChains

botao_est = navegador.find_element(By.XPATH, '/html/body/div/div/button')

# percorrendo os processos, linha por linha da tabela
for linha in processos_df.index:
    # acessando o menu de estados
    ActionChains(navegador).move_to_element(botao_est).perform()
    cidade = processos_df.loc[linha, 'Cidade']
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # indo para a nova aba que abriu
    lista_abas = navegador.window_handles
    navegador.switch_to.window(lista_abas[1])

    # preenchendo o formulario
    navegador.find_element(By.ID, 'nome').send_keys(processos_df.loc[linha, 'Nome'])
    navegador.find_element(By.ID, 'advogado').send_keys(processos_df.loc[linha, 'Advogado'])
    navegador.find_element(By.ID, 'numero').send_keys(processos_df.loc[linha, 'Processo'])

    navegador.find_element(By.XPATH, '//*[@id="formulario"]/div/button').click()

    Alert(navegador).accept()

    # aguardando a pesquisa ser feita
    while not Alert(navegador):
        time.sleep(1)
    time.sleep(5)
    
    # verificando se o processo existe e preenchendo o status da tabela
    if 'Processo encontrado com sucesso' in Alert(navegador).text:
        Alert(navegador).accept()
        processos_df.loc[linha, 'Status'] = 'Encontrado'
    else:
        Alert(navegador).accept()
        processos_df.loc[linha, 'Status'] = 'Não Encontrado'

    navegador.close()

    navegador.switch_to.window(lista_abas[0])

In [107]:
navegador.quit()

display(processos_df)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não Encontrado
3,Carol,Amanda,PC5197,São Paulo,Não Encontrado
